In [1]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
import h5py
from oasis.functions import deconvolve
from oasis import oasisAR1, oasisAR2
import pyute as ut

from importlib import reload
reload(ut)
import scipy.ndimage.filters as sfi
import scipy.stats as sst
import scipy.ndimage.measurements as snm
from mpl_toolkits.mplot3d import Axes3D
import size_contrast_analysis as sca
reload(sca)
import retinotopy_analysis as rt
reload(rt)
import naka_rushton_analysis as nra

/home/mossing/Documents/code/downloads/OASIS/oasis/functions.py:14: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  "just not the slower interior point methods we compared to in the papers.")


In [2]:
folds = []
files = []
rets = []
adjust_fns = []
rgs = []
criteria = []

def tack_on(thisfold,thisfile,retnumber,frame_adjust=None,rg=(1,0),criterion=lambda x: np.abs(x)>100):
    folds.append(thisfold)
    files.append(thisfile)
    rets.append(retnumber)
    adjust_fns.append(frame_adjust)
    rgs.append(rg)
    criteria.append(criterion)

thisfold = '180530/M8174/'
thisfile = 'M8174_130_003'
retnumber = '002'
criterion = lambda x: np.abs(x)>100
rg = (1,0)
tack_on(thisfold,thisfile,retnumber,rg=rg,criterion=criterion)

thisfold = '180220/M7254/'
thisfile = 'M7254_150_005'
retnumber = '000'
tack_on(thisfold,thisfile,retnumber)

thisfold = '180216/M7307/'
thisfile = 'M7307_120_003'
retnumber = '000'
rg = (1,-1)
tack_on(thisfold,thisfile,retnumber,rg=rg)


In [ ]:
reload(sca)
soriavg,strialavg,lb,ub,pval,nbydepth,spont,ret_vars,Smean_stat,proc = sca.analyze_everything(folds,files,rets,adjust_fns,rgs,criteria=criteria)

In [ ]:
keylist = list(soriavg.keys())

In [ ]:
ontarget = {}
for key in keylist:
    ontarget[key] = np.logical_and(soriavg[key][:,0,-1]>soriavg[key][:,0,0],np.nanmin(pval[key],axis=1)<0.01/(np.logical_not(np.isnan(pval[key][0])).sum()))
    print(ontarget[key].sum())

In [ ]:
reload(sca)
sanglemax = {}
lkatd = {}
for k in ((1,2)):
    lkatd[keylist[k]] = ontarget[keylist[k]]
def align_to_max(arr,arr0):
    arrc = np.zeros_like(arr[:,0,:])
    for i in range(arr.shape[0]):
        amy = np.argmax(arr[i].mean(-1))
        amx = np.argmax(arr0[i,amy])
        arrc[i] = np.roll(arr[i,amy],-amx)
    return arrc
def compute_sangle(thisproc,thislkat,slc=slice(-2,None)):
    strialwise = thisproc['strialwise'][thislkat]
    angle = thisproc['angle']
    size = thisproc['size']
    contrast = thisproc['contrast']
    trialrun = thisproc['trialrun']
    uangle = np.unique(angle)
    usize = np.unique(size)
    ucontrast = np.unique(contrast)
    sangle = np.zeros((thislkat.sum(),usize.size,uangle.size))
    sangle0 = np.zeros((thislkat.sum(),usize.size,uangle.size))
    for i in range(sangle.shape[1]):
        for j in range(sangle.shape[2]):
            thiscond = np.logical_and(trialrun,np.logical_and(np.in1d(contrast,ucontrast[slc]),np.logical_and(angle==uangle[j],size==usize[i])))
            sangle0[:,i,j] = strialwise[:,thiscond,4:-4][:,:1].mean(-1).mean(-1)
            sangle[:,i,j] = strialwise[:,thiscond,4:-4][:,1:].mean(-1).mean(-1)
    return sangle,sangle0

plt.figure()

plt.plot(0,0,c='k')
plt.plot(0,0,c=np.array((1,1,1))*0.6)

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='k')
plt.ylim((0,1))

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(1,4))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c=0.6*np.array((1,1,1)))
plt.ylim((0,1))
plt.legend(['PC high contrast','PC low contrast'])

plt.ylabel('norm. evoked event rate')
plt.xlabel('direction (deg.)')

# plt.savefig('pc_ori_tuning.pdf')
# plt.savefig('pc_ori_tuning.eps')

In [ ]:
reload(sca)
sanglemax = {}
lkatd = {}
for k in ((1,2)):
    lkatd[keylist[k]] = ontarget[keylist[k]]
def align_to_max(arr,arr0):
    arrc = np.zeros_like(arr[:,0,:])
    for i in range(arr.shape[0]):
        amy = np.argmax(arr[i].mean(-1))
        amx = np.argmax(arr0[i,amy])
        arrc[i] = np.roll(arr[i,amy],-amx)
    return arrc
def compute_sangle(thisproc,thislkat,slc=slice(-2,None)):
    strialwise = thisproc['strialwise'][thislkat]
    angle = thisproc['angle']
    size = thisproc['size']
    contrast = thisproc['contrast']
    trialrun = thisproc['trialrun']
    uangle = np.unique(angle)
    usize = np.unique(size)
    ucontrast = np.unique(contrast)
    sangle = np.zeros((thislkat.sum(),usize.size,uangle.size))
    sangle0 = np.zeros((thislkat.sum(),usize.size,uangle.size))
    for i in range(sangle.shape[1]):
        for j in range(sangle.shape[2]):
            thiscond = np.logical_and(trialrun,np.logical_and(np.in1d(contrast,ucontrast[slc]),np.logical_and(angle==uangle[j],size==usize[i])))
            sangle0[:,i,j] = strialwise[:,thiscond,4:-4][:,:1].mean(-1).mean(-1)
            sangle[:,i,j] = strialwise[:,thiscond,4:-4][:,1:].mean(-1).mean(-1)
    return sangle,sangle0

plt.figure(figsize=(5,5))

# plt.subplot(1,2,1)

plt.plot(0,0,c='c')
# plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='c')
plt.ylim((0,1.1))

plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train=test'])#,r'train$\neq$test'])

plt.savefig('ori_tuning_naive1.pdf')
plt.savefig('ori_tuning_naive1.eps')

plt.figure(figsize=(5,5))

# plt.subplot(1,2,1)

plt.plot(0,0,c='c')
plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='c')
plt.ylim((0,1.1))

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='b')
plt.ylim((0,1.1))

plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train=test',r'train$\neq$test'])

plt.savefig('ori_tuning_naive_and_crossval1.pdf')
plt.savefig('ori_tuning_naive_and_crossval1.eps')

plt.figure(figsize=(5,5))

# plt.subplot(1,2,2)

plt.plot(0,0,c='g')
plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle0,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='g')
plt.ylim((0,1.1))

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='b')
plt.ylim((0,1.1))

# for k in (1,2):
#     sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(1,4))
#     sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
# snorm = np.vstack([val for val in sanglemax.values()])
# snorm = np.roll(snorm,2,axis=1)
# snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
# lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
# ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c=0.6*np.array((1,1,1)))
# plt.ylim((0,1))
plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train',r'test'])

# plt.ylabel('norm. evoked event rate')
# plt.xlabel('direction (deg.)')

plt.savefig('ori_tuning_crossval_train_and_test1.pdf')
plt.savefig('ori_tuning_crossval_train_and_test1.eps')

In [ ]:
reload(sca)
sanglemax = {}
lkatd = {}
for k in ((1,2)):
    lkatd[keylist[k]] = ontarget[keylist[k]]
def align_to_max(arr,arr0):
    arrc = np.zeros_like(arr[:,0,:])
    for i in range(arr.shape[0]):
        amy = np.argmax(arr[i].mean(-1))
        amx = np.argmax(arr0[i,amy])
        arrc[i] = np.roll(arr[i,amy],-amx)
    return arrc
def compute_sangle(thisproc,thislkat,slc=slice(-2,None)):
    strialwise = thisproc['strialwise'][thislkat]
    angle = thisproc['angle']
    size = thisproc['size']
    contrast = thisproc['contrast']
    trialrun = thisproc['trialrun']
    uangle = np.unique(angle)
    usize = np.unique(size)
    ucontrast = np.unique(contrast)
    sangle = np.zeros((thislkat.sum(),usize.size,uangle.size))
    sangle0 = np.zeros((thislkat.sum(),usize.size,uangle.size))
    for i in range(sangle.shape[1]):
        for j in range(sangle.shape[2]):
            thiscond = np.logical_and(trialrun,np.logical_and(np.in1d(contrast,ucontrast[slc]),np.logical_and(angle==uangle[j],size==usize[i])))
            sangle0[:,i,j] = strialwise[:,thiscond,4:-4][:,:3].mean(-1).mean(-1)
            sangle[:,i,j] = strialwise[:,thiscond,4:-4][:,3:].mean(-1).mean(-1)
    return sangle,sangle0

plt.figure(figsize=(5,5))

# plt.subplot(1,2,1)

plt.plot(0,0,c='c')
# plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='c')
plt.ylim((0,1.1))

plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train=test'])#,r'train$\neq$test'])

plt.savefig('ori_tuning_naive.pdf')
plt.savefig('ori_tuning_naive.eps')

plt.figure(figsize=(5,5))

# plt.subplot(1,2,1)

plt.plot(0,0,c='c')
plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='c')
plt.ylim((0,1.1))

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='b')
plt.ylim((0,1.1))

plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train=test',r'train$\neq$test'])

plt.savefig('ori_tuning_naive_and_crossval.pdf')
plt.savefig('ori_tuning_naive_and_crossval.eps')

plt.figure(figsize=(5,5))

# plt.subplot(1,2,2)

plt.plot(0,0,c='g')
plt.plot(0,0,c='b')

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle0,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='g')
plt.ylim((0,1.1))

for k in (1,2):
    sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(-2,None))
    sanglemax[keylist[k]] = align_to_max(sangle,sangle0)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
snorm = np.vstack([val for val in sanglemax.values()])
snorm = np.roll(snorm,2,axis=1)
snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c='b')
plt.ylim((0,1.1))

# for k in (1,2):
#     sangle,sangle0 = compute_sangle(proc[keylist[k]],lkatd[keylist[k]],slc=slice(1,4))
#     sanglemax[keylist[k]] = align_to_max(sangle,sangle)-spont[keylist[k]][lkatd[keylist[k]]][:,np.newaxis]
# snorm = np.vstack([val for val in sanglemax.values()])
# snorm = np.roll(snorm,2,axis=1)
# snorm = np.hstack((snorm,snorm[:,0:1]))/(snorm.max(1)[:,np.newaxis])
# lb,ub = ut.bootstrap(snorm,np.nanmean,axis=0,pct=(2.5,97.5))
# ut.plot_errorbar_hillel(45*np.arange(9),np.nanmean(snorm,0),lb,ub,c=0.6*np.array((1,1,1)))
# plt.ylim((0,1))
plt.xlabel('direction (deg.)')
plt.ylabel('norm. event rate')
plt.legend([r'train',r'test'])

# plt.ylabel('norm. evoked event rate')
# plt.xlabel('direction (deg.)')

plt.savefig('ori_tuning_crossval_train_and_test.pdf')
plt.savefig('ori_tuning_crossval_train_and_test.eps')